# 获取holiday的多种方法

## 公共字段

In [3]:
import csv
import requests
from lxml import etree
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os

""" holiday1 : 表示是假期，即包含周末，节假日 """
col = ["t","year","weekofday","holiday1","holiday2", "tx"]
def get_time_feat(df, ts='t'):
    df[ts] = pd.to_datetime(df[ts])
    df['year'] = df[ts].dt.year
    df['dayofweek'] = df[ts].dt.dayofweek
    df['dayofweek'] = [i+1 for i in df['dayofweek']]
    return df 

if os.path.exists('Data'):
    pass
else:
    os.mkdir('Data')

## 方法1X-万年历

https://blog.51cto.com/u_16099200/6561976

In [398]:
import csv
import requests
from lxml import etree
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def m1_get_year_holiday(year):
    url = 'https://wannianrili.51240.com/ajax/'
    s = requests.session()
    result = []
    # 生成月份列表
    dateList = [year + '-' + '%02d' % i for i in range(1, 13)]
    for year_month in dateList:
        s = requests.session()
        url = 'https://wannianrili.51240.com/ajax/'
        payload = {'q': year_month}
        response = s.get(url, params=payload)
        element = etree.HTML(response.text)
        html = element.xpath('//div[@class="wnrl_riqi"]')
        # print('In Working:', year_month)
        for _element in html:
            # 获取节点属性
            item = _element.xpath('./a')[0].attrib
            # print(item)
            if 'class' in item:
                if item['class'] == 'wnrl_riqi_xiu':
                    tag = '假期休息'
                elif item['class'] == 'wnrl_riqi_ban':
                    tag = '补班'
                elif item['class'] == 'wnrl_riqi_mo':
                    tag = '周末休息'
                else:
                    pass
                _span = _element.xpath('.//text()')
                result.append({'Date': year_month + '-' + _span[0], 'Holiday': _span[1], 'Tag': tag})
            else:
                _span = _element.xpath('.//text()')
                result.append({'Date': year_month + '-' + _span[0], 'Holiday': _span[1], 'Tag': '工作日'})
    df = pd.DataFrame.from_dict(result)
    return df

def get_m1_all():
    df = pd.DataFrame()
    for year in range(2018, 2025):
        tmp = m1_get_year_holiday(str(int(year)))
        df = df.append(tmp, ignore_index=True )
    df.columns = ['t','day','name']
    df['t'] = pd.to_datetime(df['t'])
    df['dayofweek']=df['t'].dt.dayofweek
    df['dayofweek']=[i+1 for i in df['dayofweek']]
    df['tx'] = 0    # 是否调休
    df['holiday'] = 0 # 是否节假日

    df.loc[df['name'].str.contains('补班'),'tx'] = 1
    df.loc[df['name'].str.contains('假期'),'holiday'] = 1
    df['year'] = df['t'].dt.year
    return df

# df1 = get_m1_all() # 不用再执行，因为23年及其之后没有数据 
# df1.to_csv('Data/df1.csv', index=None ) # 只有2018年到2022年的数据 


In [413]:
for i in df1.columns:
    print(i, df1[i].nunique(), '   ', df1[i].unique())

t 1826     ['2018-01-01T00:00:00.000000000' '2018-01-02T00:00:00.000000000'
 '2018-01-03T00:00:00.000000000' ... '2022-12-29T00:00:00.000000000'
 '2022-12-30T00:00:00.000000000' '2022-12-31T00:00:00.000000000']
day 110     ['元旦' '十六' '十七' '十八' '小寒' '二十' '廿一' '廿二' '廿三' '廿四' '廿五' '廿六' '廿七' '廿八'
 '廿九' '三十' '十二月' '初二' '初三' '大寒' '初五' '初六' '初七' '腊八节' '初九' '初十' '十一' '十二'
 '十三' '十四' '十五' '立春' '北小年' '南小年' '情人节' '除夕' '春节' '雨水' '初八' '元宵节' '爱耳日'
 '惊蛰' '十九' '妇女节' '植树节' '315' '二月' '龙抬头' '初四' '睡眠日' '安全日' '愚人节' '清明' '三月'
 '谷雨' '地球日' '劳动节' '青年节' '立夏' '母亲节' '四月' '电信日' '520' '小满' '儿童节' '环境日' '芒种'
 '五月' '父亲节' '端午节' '夏至' '禁毒日' '建党节' '小暑' '六月' '大暑' '建军节' '立秋' '七月' '七夕'
 '处暑' '中元节' '白露' '教师节' '国耻日' '秋分' '中秋节' '国庆节' '寒露' '九月' '粮食日' '重阳节' '霜降'
 '万圣节' '立冬' '十月' '双十一' '感恩节' '大雪' '双十二' '公祭日' '冬至' '平安夜' '圣诞节' '618' '八月'
 '小雪' '十一月' '春分' '闰四月']
name 4     ['假期休息' '工作日' '周末休息' '补班']
dayofweek 7     [1 2 3 4 5 6 7]
tx 1     [0]
holiday 2     [1 0]
year 5     [2018 2019 2020 2021 2022]


## 方法2-假日工具包chinese_calendar

In [88]:
# 是否节假日，是否工作日，是否调休 
from chinese_calendar import is_holiday, is_workday, is_in_lieu 

"""
workday 》1） 周一到周五正常上班
        》2）周末调休补班，weekday = 6,7

"""
def m2_get_year_holiday(df):
    df['year'] = df['t'].dt.year
    df['holiday'] = [1 if is_holiday(i) else 0 for i in df['t'] ]  # 周六周日，以及节假日,
    df['tx'] = [1 if is_in_lieu(i) else 0 for i in df['t'] ]      # 节假日是否是调休 
    df['workday'] = [1 if is_workday(i) else 0 for i in df['t'] ] # 是否上班,和holiday 互反 
    df['dayofweek'] = df['t'].dt.dayofweek 
    df['dayofweek'] = [i+1 for i in df['dayofweek']]
    return df 

def get_m2(start,end):
    df2 = pd.DataFrame( pd.date_range(start, end, freq='1D'), columns=['t'])
    df2 = m2_get_year_holiday(df2 )
    return df2 

# 只需要的是holiday
star t= '2018-01-01'
end = '2024-12-31'
df2['tx2'] = 0
df2.loc[(df2['holiday']==0)&(df2['dayofweek'].isin([6,7])),'tx2'] = 1
df2.to_csv('Data/df2.csv', index=None)

In [313]:
所有放假 》 只是周末假期 》 只是节假日，至少3天的假期 


1    2557
dtype: int64

## 方法3 X-json 

https://github.com/search?q=%E8%8A%82%E5%81%87%E6%97%A5&type=repositories

In [93]:
import requests
import json

url1 = "https://www.shuyz.com/githubfiles/china-holiday-calender/master/holidayAPI.json"
url2 = "https://raw.githubusercontent.com/lanceliao/china-holiday-calender/master/holidayAPI.json"
headers = {'Accept': 'application/json'}

r = requests.get(url1)
r.json()

{'Name': '中国节假日补班日历',
 'Version': '1.0',
 'Generated': '20240103T182037Z',
 'Timezone': 'Asia/Shanghai',
 'Author': 'ShuYZ.com',
 'URL': 'https://github.com/lanceliao/china-holiday-calender',
 'Years': {'2024': [{'Name': '元旦',
    'StartDate': '2024-01-01',
    'EndDate': '2024-01-01',
    'Duration': 1,
    'CompDays': [],
    'URL': 'https://www.gov.cn/zhengce/content/202310/content_6911527.htm',
    'Memo': '一、元旦：1月1日放假，与周末连休。'},
   {'Name': '春节',
    'StartDate': '2024-02-10',
    'EndDate': '2024-02-17',
    'Duration': 8,
    'CompDays': ['2024-02-04', '2024-02-18'],
    'URL': 'https://www.gov.cn/zhengce/content/202310/content_6911527.htm',
    'Memo': '二、春节：2月10日至17日放假调休，共8天。2月4日（星期日）、2月18日（星期日）上班。鼓励各单位结合带薪年休假等制度落实，安排职工在除夕（2月9日）休息。'},
   {'Name': '清明节',
    'StartDate': '2024-04-04',
    'EndDate': '2024-04-06',
    'Duration': 3,
    'CompDays': ['2024-04-07'],
    'URL': 'https://www.gov.cn/zhengce/content/202310/content_6911527.htm',
    'Memo': '三、清明节：4月4日至6日放假调休，共3天。4月7日（星

## 方法4 X-国务院发布

In [124]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import requests
import re
from bs4 import BeautifulSoup
import datetime
import time

def get_url_html(url, encoding=None):
    r = requests.get(url)
    print(url)
    if not encoding:
        encoding = r.encoding
    return r.content.decode(encoding).encode('utf-8')


def get_holiday_url(year, html_doc):
    soup = BeautifulSoup(html_doc, "lxml")
    td_list = soup.findAll("td", {"class": "info"})
    for td in td_list:
        link = td.find("a")
        title = "".join(link.contents)
        search = re.search(u'国务院办公厅关于(\d+)年部分节假日安排的通知', title)
        if search and int(search.group(1)) == year:
            print(link['href'])
            return link["href"]
    return None


def match_holiday(html_doc, year=datetime.datetime.now().year + 1):
    html_doc = re.sub(r'<(/|)br(/|)>', '\n', html_doc) # 将标签换行改为换行符。
    html_doc = html_doc.replace('　', '')  # 过滤掉全角空格
    soup = BeautifulSoup(html_doc, "lxml")
    td_content = soup.find("td", {"class": "b12c"})
    html = td_content.get_text()
    html = html.split(u"\n")
    holiday = []
    for line in html:
        mat = re.search(u"^[一二三四五六七八九十]*?、(.*?)$", line.strip())
        if mat:
            desc = mat.group(1)
            name, plan = desc.split(u"：", 2)
            holiday_mat = re.search(u"(?P<holiday>\d+月\d+日.*?)放假", plan)
            workday_mat = re.search(u"[，。](?P<workday>\d+月\d+日[^，。]*?)上班", plan)
            holiday_desc = holiday_mat.group("holiday") if holiday_mat else ""
            workday_desc = workday_mat.group("workday") if workday_mat else ""
            holiday_parse = parse_date(holiday_desc, year)
            workday_parse = parse_date(workday_desc, year)
            if not holiday_parse:
                raise Exception(u"解析{}年节日{}假期失败！".format(year, name))
            holiday.append({
                "name": name,
                "holiday": holiday_parse,
                "workday": workday_parse,
            })

    return holiday


def parse_date(desc, year):
    mat_days = re.findall(
        u"(?P<from_month>\d+)月(?P<from_day>\d+)日至(?P<to_month>\d+)月(?P<to_day>\d+)日", desc)
    if mat_days:
        days = []
        month_day = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        if (year % 4 == 0 and year % 100 != 0) or year % 400 == 0:
            month_day[2] = 29
        for mat in mat_days:
            from_month, from_day, to_month, to_day = mat
            day_begin = from_day
            for month in range(int(from_month), int(to_month) + 1):
                if month == int(to_month):
                    day_end = to_day
                else:
                    day_end = month_day[month - 1]
                for day in range(int(day_begin), int(day_end) + 1):
                    days.append(get_time_stamp(year, month, day))
                day_begin = 1
        return days
    mat_days = re.findall(u"(?P<month>\d+)月(?P<from>\d+)日至(?P<to>\d+)日", desc)
    if mat_days:
        days = []
        for mat in mat_days:
            month, from_day, to_day = mat
            for day in range(int(from_day), int(to_day) + 1):
                days.append(get_time_stamp(year, int(month), day))
        return days
    mat_days = re.findall(u"(?P<month>\d+)月(?P<from>\d+)日", desc)
    if mat_days:
        days = []
        for mat in mat_days:
            month, day = mat
            days.append(get_time_stamp(year, int(month), int(day)))
        return days

    return []

def get_time_stamp(year, month, day):
    return int(time.mktime(
        datetime.datetime(
            year, month, day, 0, 0, 0, 0
        ).timetuple()
    ))

def get_holiday(year):
    url = "http://sousuo.gov.cn/list.htm?q=&n=15&t=paper&childtype=&subchildtype=gc189&pcodeJiguan=%E5%9B%BD%E5%8A%9E%E5%8F%91%E6%98%8E%E7%94%B5&pcodeYear=&pcodeNum=&location=%25E7%25BB%25BC%25E5%2590%2588%25E6%2594%25BF%25E5%258A%25A1%25E5%2585%25B6%25E4%25BB%2596&sort=pubtime&searchfield=title%3Acontent%3Apcode%3Apuborg%3Akeyword&title=&content=&pcode=&puborg=&timetype=timeqb&mintime=&maxtime="    
    print(url)
#     holiday_url = get_holiday_url(year, get_url_html(url))
#     if holiday_url:
#         return match_holiday(get_url_html(holiday_url, 'utf-8'), year)
#     return None

df4 = get_holiday(datetime.datetime.now().year)


http://sousuo.gov.cn/list.htm?q=&n=15&t=paper&childtype=&subchildtype=gc189&pcodeJiguan=%E5%9B%BD%E5%8A%9E%E5%8F%91%E6%98%8E%E7%94%B5&pcodeYear=&pcodeNum=&location=%25E7%25BB%25BC%25E5%2590%2588%25E6%2594%25BF%25E5%258A%25A1%25E5%2585%25B6%25E4%25BB%2596&sort=pubtime&searchfield=title%3Acontent%3Apcode%3Apuborg%3Akeyword&title=&content=&pcode=&puborg=&timetype=timeqb&mintime=&maxtime=


## 方法5-百度日历

In [298]:
import pandas as pd
import requests
import json
import numpy as np
import re

# 生成某一年的日历
def gen_calendar(year=2022):
    df=pd.DataFrame()
    # 获取法定节假日
    up1='https://sp1.baidu.com/8aQDcjqpAAV3otqbppnN2DJv/api.php?tn=wisetpl&format=json&resource_id=39043&query='
    up2='月&t=1642579711570&cb=op_aladdin_callback1642579711570'
    # 2022年接口已经改为按月获取数据，因此循环12个月获取每月数据
    for i in range(1,13):
        url="".join([up1,str(year),"年",str(i),up2])
        #print(url)
        r=requests.get(url)
        rtxt=re.split("[()]", r.text)[1]
        r_json=json.loads(rtxt)
        each_d1=pd.DataFrame(r_json['data'])
        each_d2=pd.DataFrame(each_d1['almanac'][0])
        ## 筛选当月数据，数据中year\month\day是公立日期，数据会返回3个月的数据
        each_d3=each_d2[each_d2['month']==str(i)]
        # ## 由于12月的数据没有status字段（可能是下一年一月放假规定没出来，所以这里进行特殊处理)--结论应该是某些月份没有节假日就没有status字段，所以后面专门针对status进行处理
        # if i==11:
        #     each_d3=each_d2[each_d2['month']>=str(i)]
        # else:
        #     each_d3=each_d2[each_d2['month']==str(i)]
        ## 组合真的日期，并标记节假日情况
        each_d3['公历日期']=pd.to_datetime(each_d3['year']+"/"+each_d3['month']+"/"+each_d3['day'])
        each_d3['周几']=each_d3['公历日期'].dt.dayofweek+1
        ## 标记是否节假日(即当天是否是法定放假)，其中status中的1表示放假，2表示上班，注意需要处理有没有status列的情况
        ## 有status的情况
        if "status" in each_d3.columns:
            each_d3['status'].fillna(0,inplace=True)
            each_d3['status']=each_d3['status'].astype('int',errors='ignore')
            ## 按周几把周末标记成1，周一至周五标记成0，然后通过标记和status的值相加，结果为1和2的就是假期
            judge=np.where(each_d3['周几']<6,0,1)+each_d3['status']
            each_d3['是否节假日']=np.where((judge==1) | (judge==2),"Y","N")
            df=df.append(each_d3)
            # each_d3.to_csv("百度日历.csv",index=False)
        ## 没有status的情况，直接按周末为节假日
        else:
            each_d3['是否节假日']=np.where(each_d3['周几']>=6,"Y","N")
            df=df.append(each_d3)
    # 重命名列
    df.rename(columns={'animal':'生肖','avoid':'忌','cnDay':'中文星期','day':'日','gzDate':'干支日',
                       'gzMonth':'干支月','gzYear':'干支年','isBigMonth':'是否为阴历大月',
                       'lDate':'中文阴历日','lMonth':'中文阴历月','lunarDate':'数字阴历日',
                       'lunarMonth':'数字阴历月','lunarYear':'数字阴历年','month':'月',
                       'oDate':'阳历当天0点','suit':'宜','term':'节气节日','type':'各种与节日有关的类型',
                       'value':'各种日','year':'年','desc':'一种节日','status':'1休假2上班'},
              inplace=True)
    return df

def m5_get_year_holiday(start,end):
    df=pd.DataFrame()
    for year in range(start,end):
        each_df=gen_calendar(year=year)
        df=df.append(each_df, ignore_index=True )
    return df
    
# df5 = m5_get_year_holiday(2002,2025)
# df5.to_csv('Data/df5_all.csv',index=None )
# df5.to_csv('df.csv', index=None)
# df5t = df5.copy()

'https://sp1.baidu.com/8aQDcjqpAAV3otqbppnN2DJv/api.php?tn=wisetpl&format=json&resource_id=39043&query=2020年12月&t=1642579711570&cb=op_aladdin_callback1642579711570'

In [240]:
# cols = ["节气节日","公历日期","是否节假日"]
# df5 = df5[cols]
# df5.columns = ['name','t',  'holiday']

del df5['sb']
df5 = get_time_feat(df5,ts='t')

df5['holiday']=df5['holiday'].replace('Y',1)
df5['holiday']=df5['holiday'].replace('N',0)

# 调休 》是周末dayofweek为6，7，但是上班 即holiday=0
df5['tx'] = 0
df5.loc[(df5['dayofweek'].isin([6,7]))&(df5['holiday']==0),"tx"] = 1 

# 只能是获取
df5.to_csv('Data/df5.csv', index=None)

### 对比df25

In [348]:
dfs = pd.merge(df2, df5[['t','holiday','tx']].rename(columns={'tx':'tx5','holiday':'holiday5'}),
               how='left', on=['t']
              )

In [349]:
"""
万年历和百度日历对比 

1 df2 和df5，工作日上班的两个特征一样 
2 holiday 和 holiday5， 判断是否节假日一样 
3 

"""


display( dfs[dfs['tx2']!=dfs['tx5']] )

display( dfs[dfs['holiday']!=dfs['holiday5']] )



,t,year,holiday,tx,workday,dayofweek,tx2,holiday5,tx5


In [388]:
dfs[(dfs['tx2']!=dfs['tx']) & (dfs['tx']==dfs['holiday'] )&(dfs['t']<'2024-04-30')].tail(11)

,t,year,holiday,tx,workday,dayofweek,tx2,holiday5,tx5
2103,2023-10-05,2023,1,1,0,4,0,1,0
2104,2023-10-06,2023,1,1,0,5,0,1,0
2105,2023-10-07,2023,0,0,1,6,1,0,1
2106,2023-10-08,2023,0,0,1,7,1,0,1
2225,2024-02-04,2024,0,0,1,7,1,0,1
2236,2024-02-15,2024,1,1,0,4,0,1,0
2237,2024-02-16,2024,1,1,0,5,0,1,0
2239,2024-02-18,2024,0,0,1,7,1,0,1
2286,2024-04-05,2024,1,1,0,5,0,1,0
2288,2024-04-07,2024,0,0,1,7,1,0,1


In [382]:
dfs[dfs['tx']!=dfs['holiday'] ].shape

(760, 9)

In [383]:
dfs[dfs['tx']==dfs['holiday'] ].shape

(1797, 9)

In [112]:
df6 =  m5_get_year_holiday(2023,2025)

In [119]:
def getOfficialHolidays(year): #获取国务院节假日安排，包括调休
    url = "https://www.gov.cn/zhengce/content/202310/content_6911527.htm"  #2024年放假安排url
    rep = requests.get(url)  # Get方式获取网页数据
    rep.encoding = 'utf-8'

    soup = BeautifulSoup(rep.text, 'html.parser')
    pList = soup.find_all('p')
    for item in pList:
        p = item.text
        if '：' in p and '放假' in p:   #找出节日对应的几行内容
            print(p)
            
getOfficialHolidays(2024)


一、元旦：1月1日放假，与周末连休。
二、春节：2月10日至17日放假调休，共8天。2月4日（星期日）、2月18日（星期日）上班。鼓励各单位结合带薪年休假等制度落实，安排职工在除夕（2月9日）休息。
三、清明节：4月4日至6日放假调休，共3天。4月7日（星期日）上班。
四、劳动节：5月1日至5日放假调休，共5天。4月28日（星期日）、5月11日（星期六）上班。
五、端午节：6月10日放假，与周末连休。
六、中秋节：9月15日至17日放假调休，共3天。9月14日（星期六）上班。
七、国庆节：10月1日至7日放假调休，共7天。9月29日（星期日）、10月12日（星期六）上班。


## 方法6-便民网

In [248]:
# coding:utf-8
import requests
import re
import importlib, sys

importlib.reload(sys)
import pandas as pd
import datetime
from openpyxl import load_workbook

'''爬虫表格示例'''

# 2019年节日放假安排查询
# 节日	放假时间	调休上班日期	放假天数
# 元旦	12月30日~1月1日	12月29日（周六）上班	3天
# 春节	2月4日(除夕）~2月10日	2月2日（周六）、2月3日（周日）上班	7天
# 清明节	4月5日~4月7日	与周末连休	3天
# 劳动节	5月1日~5月4日	4月28日（星期日）、5月5日（星期日）上班	4天
# 端午节	6月7日~6月9日	与周末连休	3天
# 国庆节	10月1日~10月7日	9月29日（周日）、10月12日（周六）上班	7天
# 中秋节	9月13日~9月15日	与周末连休	3天

'''适用条件'''
# 1. 放假时间列必须是以xx月xx日开始，且以~分隔的连续日期
# 2. 调休上班日期列中，包含“休”“无”那行没有上班日期安排

class spider(object):
    def __init__(self):
        print(u'开始爬取...')

    # 获取网页源码，返回页面所有信息
    def getSource(self, url):
        html = requests.get(url)
        return html.text

    # 改变url实现抓取所有年份的假日，得所有url列表
    def changePage(self, url, now_page, total_page):
        page_group = []
        for i in range(now_page, total_page + 1):
            link = url[:26] + str(i) + url[30:]
            page_group.append(link)
        return page_group

    # 把页面上假日的table截取出来，返回字符串
    def geteveyTable(self, source):
        everytable = re.findall('(<table style=.*?</table>)', source, re.S)
        return everytable

    # 取出放假日期范围，返回日期列表
    def getHoliday(self, year, string):
        string = "".join(re.findall('\d.*', string, re.S))
        if string.find('年') >= 0:
            string = string[5:]
        if string.find('~') == -1:
            string = string + '~' + string
        str1 = string.split('~')
        m1 = "".join(re.findall('\d.*(?=月\d)', str1[0], re.S))
        d1 = "".join(re.findall('(?<=月).*\d', str1[0], re.S))
        m2 = "".join(re.findall('\d.*(?=月\d)', str1[1], re.S))
        d2 = "".join(re.findall('(?<=月).*\d', str1[1], re.S))
        # 规范化日期格式为yyyy mm dd
        if int(m1) < 10:
            m1 = '0' + m1

        if int(d1) < 10:
            d1 = '0' + d1

        if int(m2) < 10:
            m2 = '0' + m2

        if int(d2) < 10:
            d2 = '0' + d2

        if int(m1) == 12:
            y1 = str(int(year) - 1)  # 如果元旦前是12月则月份要取少一年
        else:
            y1 = year
        # 根据日期范围获取所有连续日期
        date_list = []
        begin_date = datetime.datetime.strptime(y1 + m1 + d1, "%Y%m%d")
        end_date = datetime.datetime.strptime(year + m2 + d2, "%Y%m%d")
        while begin_date <= end_date:
            date_str = begin_date.strftime("%Y%m%d")
            date_list.append(date_str)
            begin_date += datetime.timedelta(days=1)

        return date_list

    # 取出调休上班日期，返回日期列表
    def getWorkday(self, year, string):
        string = "".join(re.findall('\d.*', string, re.S))
        if string.find('年') >= 0:
            string = string[5:]
        list1 = re.split('[，、]', string)
        list2 = []
        print('list1 ',list1)
        for a in list1:
           
            m1 = "".join(re.findall('\d.*(?=月\d)', a, re.S))
            d1 = "".join(re.findall('(?<=月).*\d', a, re.S))
            
            if int(m1) == 12:
                y1 = str(int(year) - 1)
            else:
                y1 = year
            if int(m1) < 10:
                m1 = '0' + m1
            if int(d1) < 10:
                d1 = '0' + d1

            list2.append(y1 + m1 + d1)

        return list2

    # 分割源码获得每年所有假日表：日期、节日名称、是否放假
    def getInfo(self, eachyear):
        info = {}
        ho = re.findall('jiad/">(.*?)</a>', eachyear, re.S)  # 节日名称列表
        year = re.search('<a href="/(.*?)_', eachyear, re.S).group(1)  # 年份字符串
        a = re.findall('<td>(.*?)</td>', eachyear, re.S)  # 除了节日名称列的其他数据列表，包括放假时间、调休上班日期、放假天数
        j = len(a)
        # 得出放假日期
        i = 0
        k = 0
        df = pd.DataFrame()
        while (i < j):
            if i > j:
                continue
            holidayname = ''.join(ho[k])
            holiday_list = list(map(int, self.getHoliday(year, a[i])))
            df1 = pd.DataFrame(holiday_list, columns=['period_id'])
            df1['holiday_name'], df1['is_holiday'] = [holidayname, 1]
            df = pd.concat([df, df1])
            i += 3
            k += 1
        # 得出调休上班日期
        m = 1
        n = 0
        df2 = pd.DataFrame()
        while (m < j):
            if m > j:
                continue
            holidayname = ''.join(ho[n])
            data = a[m]
            checkdata = a[m].find('休')  # 是否包含休
            checkdata2 = a[m].find('无')  # 是否包含无
            m += 3
            n += 1
            if checkdata >= 0 or checkdata2 >= 0:  # 如果该节日没有特殊调休安排则跳过循环
                continue
            work_list = list(map(int, self.getWorkday(year, data)))
            df_temp = pd.DataFrame(work_list, columns=['period_id'])
            df_temp['holiday_name'], df_temp['is_holiday'] = [holidayname, 0]
            df2 = pd.concat([df2, df_temp])
        return pd.concat([df, df2])

    # 将结果写入excel
    def write2excel(self, excelWriter, year, data):
        excel_header = list(data.columns.values)  # excel的标题
        data.to_excel(excelWriter, sheet_name=year, header=excel_header, index=False)
        writer.save()
        writer.close()

    # 将结果追加到excel
    def add2excel(self, excelWriter, year, data):
        book = load_workbook(excelWriter.path)
        excelWriter.book = book
        excel_header = list(data.columns.values)  # excel的标题
        data.to_excel(excelWriter, sheet_name=year, header=excel_header, index=False)
        excelWriter.save()
        excelWriter.close()

# 初始化参数
url = 'https://fangjia.51240.com/2024__fangjia/'  # 抓取的页面
start_page = 2002  # 抓取开始年份
end_page = 2024  # 抓取结束年份
filepath = '节假日安排爬虫数据.xlsx'  # 写入excel文件路径
print(u'初始化类..')
myspider = spider()
all_links = myspider.changePage(url, start_page, end_page)
df = pd.DataFrame()
for link in all_links:
    print(u'正在处理页面..' + link)
    try:
        html = myspider.getSource(link)
        everyyear = myspider.geteveyTable(html)
        for each in everyyear:
            df = pd.concat([df, myspider.getInfo(each)])
    except:
        print('wrong ..... ' )
df.index = range(len(df))
df.sort_values(by='period_id', inplace=True)

#     # 创建ExcelWriter对象
#     writer = pd.ExcelWriter(filepath, engine='openpyxl')

#     # sheet名
#     if start_page == end_page:
#         sheetname = str(end_page)
#     else:
#         sheetname = str(start_page) + '-' + str(end_page)

#     # 生成excel
#     myspider.write2excel(writer, sheetname, df)  # 写入excel，会覆盖掉所有已存在的sheet

#     # excel中增加sheet
#     # myspider.add2excel(writer, sheetname, df)  # 写入excel，并新增sheet

#     # 读取写入结果
#     # data = pd.read_excel('data/节假日安排爬虫数据.xlsx', sheet_name=str(end_page))
#     # print(data)
#     print('写入Excel完成！')


初始化类..
开始爬取...
正在处理页面..https://fangjia.51240.com/2002__fangjia/
正在处理页面..https://fangjia.51240.com/2003__fangjia/
正在处理页面..https://fangjia.51240.com/2004__fangjia/
正在处理页面..https://fangjia.51240.com/2005__fangjia/
正在处理页面..https://fangjia.51240.com/2006__fangjia/
正在处理页面..https://fangjia.51240.com/2007__fangjia/
正在处理页面..https://fangjia.51240.com/2008__fangjia/
正在处理页面..https://fangjia.51240.com/2009__fangjia/
正在处理页面..https://fangjia.51240.com/2010__fangjia/
正在处理页面..https://fangjia.51240.com/2011__fangjia/
正在处理页面..https://fangjia.51240.com/2012__fangjia/
list1  ['12月31日（星期六）']
list1  ['1月21日（星期六）', '1月29日（星期日）']
list1  ['3月31日（星期六）', '4月1日（星期日）']
list1  ['4月28日（星期六）']
list1  ['9月29日（星期六）']
正在处理页面..https://fangjia.51240.com/2013__fangjia/
list1  ['1月5日（周六）', '1月6日（周日）']
list1  ['2月16日（周六）', '2月17日（周日）']
list1  ['4月7日（周日）']
list1  ['4月27日（周六）', '4月28日（周日）']
list1  ['6月8日（周六）', '6月9日（周日）']
list1  ['9月22日（周日）']
list1  ['9月29日（周日）', '10月12日（周六）']
正在处理页面..https://fangjia.51240.com/2014__fangjia/
li

In [275]:
df['t'] = [datetime.datetime.strptime(str(i),'%Y%m%d') for i in df['period_id']]
df = df.reset_index(drop=True)
df = get_time_feat(df)
d1 = df.groupby('year')['t'].count().reset_index()
a1 = list(range(d1.year.min(), d1.year.max()+1 ))


,year,t
0,2011,1
1,2012,33
2,2013,41
3,2014,32
4,2015,32
5,2016,36
6,2017,42
7,2018,36
8,2019,34
9,2021,38


(287, 3)

In [244]:
df = pd.read_csv('df.csv')
df.tail()

,生肖,忌,中文星期,日,festivalList,干支日,干支月,干支年,是否为阴历大月,中文阴历日,...,各种日,年,yjJumpUrl,yj_from,一种节日,1休假2上班,festival,公历日期,周几,是否节假日
2552,龙,安葬.行丧.伐木.斋醮.作梁,五,27,NaN,乙丑,丙子,甲辰,1.0,廿七,...,NaN,2024,https://mobile.51wnl-cq.com/huangli_tab_h5/?po...,51wnl,NaN,NaN,NaN,2024-12-27,5,N
2553,龙,结婚.搬新房.动土.祈福.出火.破土,六,28,NaN,丙寅,丙子,甲辰,1.0,廿八,...,NaN,2024,https://mobile.51wnl-cq.com/huangli_tab_h5/?po...,51wnl,NaN,NaN,NaN,2024-12-28,6,Y
2554,龙,结婚.合婚订婚.搬新房.订盟.掘井,日,29,NaN,丁卯,丙子,甲辰,1.0,廿九,...,NaN,2024,https://mobile.51wnl-cq.com/huangli_tab_h5/?po...,51wnl,NaN,NaN,NaN,2024-12-29,7,Y
2555,龙,买房.开业.掘井.造庙,一,30,二九天,戊辰,丙子,甲辰,1.0,三十,...,NaN,2024,https://mobile.51wnl-cq.com/huangli_tab_h5/?po...,51wnl,NaN,NaN,NaN,2024-12-30,1,N
2556,龙,安葬.开生坟.行丧.破土,二,31,NaN,己巳,丙子,甲辰,NaN,初一,...,NaN,2024,https://mobile.51wnl-cq.com/huangli_tab_h5/?po...,51wnl,NaN,NaN,NaN,2024-12-31,2,N


In [ ]:
def exportCSV(data):
    """导出CSV"""
    headers = ['Date', 'Holiday', 'Tag']
    # 如果存入乱码，添加 encoding='utf-8-sig'
    with open(year + 'Holiday.csv', 'w', newline='')as f:
        f_csv = csv.DictWriter(f, headers)
        f_csv.writeheader()
        f_csv.writerows(data)a
exportCSV(result)

# 周末or假期判断

In [232]:
def if_festival(df2):
    df2['t'] = pd.to_datetime(df2['t'])

    df = pd.DataFrame() 
    df['t'] = pd.date_range(df2.t.min(), df2.t.max(), freq='1D')
    df['a1'] = 1

    df = pd.merge(df, df2[['t','holiday']], how='left', on=['t'])
    df['a2'] = 1-df['holiday']
    df['a3'] = df['a2'].cumsum()

    df = pd.merge( df, 
              df[df['holiday']==1].groupby('a3')['t'].count().reset_index().rename(columns={'t':'holidays'}),
              how='left',on=['a3'])
    df.fillna(0,inplace=True)        
    df['festival'] = 0 
    df.loc[(df['holiday']==1 )&( df['holidays']>2), 'festival']=1
    return df[['t','holiday','festival','a3']]    

# df2 要有时间，假期即holiday, 
data = pd.read_csv('Data/df2.csv')
data = if_festival(data)

,t,holiday,festival,a3
0,2018-01-01,1,0,0
1,2018-01-02,0,0,1
2,2018-01-03,0,0,2
3,2018-01-04,0,0,3
4,2018-01-05,0,0,4
...,...,...,...,...
2552,2024-12-27,0,0,1746
2553,2024-12-28,1,0,1746
2554,2024-12-29,1,0,1746
2555,2024-12-30,0,0,1747


[[1, 17000],
 [2, 34000],
 [3, 51000],
 [4, 68000],
 [5, 85000],
 [6, 102000],
 [7, 119000],
 [8, 136000],
 [9, 153000],
 [10, 170000],
 [11, 187000]]

785

In [182]:
for i, f in df[df.festival==1].groupby('a3'):
    display(f.head(10))

,t,a1,holiday,a2,a3,holidays,festival
45,2018-02-15,1,1,0,33,7.0,1
46,2018-02-16,1,1,0,33,7.0,1
47,2018-02-17,1,1,0,33,7.0,1
48,2018-02-18,1,1,0,33,7.0,1
49,2018-02-19,1,1,0,33,7.0,1
50,2018-02-20,1,1,0,33,7.0,1
51,2018-02-21,1,1,0,33,7.0,1


,t,a1,holiday,a2,a3,holidays,festival
94,2018-04-05,1,1,0,64,3.0,1
95,2018-04-06,1,1,0,64,3.0,1
96,2018-04-07,1,1,0,64,3.0,1


,t,a1,holiday,a2,a3,holidays,festival
118,2018-04-29,1,1,0,81,3.0,1
119,2018-04-30,1,1,0,81,3.0,1
120,2018-05-01,1,1,0,81,3.0,1


,t,a1,holiday,a2,a3,holidays,festival
166,2018-06-16,1,1,0,114,3.0,1
167,2018-06-17,1,1,0,114,3.0,1
168,2018-06-18,1,1,0,114,3.0,1


,t,a1,holiday,a2,a3,holidays,festival
264,2018-09-22,1,1,0,183,3.0,1
265,2018-09-23,1,1,0,183,3.0,1
266,2018-09-24,1,1,0,183,3.0,1


...

,t,a1,holiday,a2,a3,holidays,festival
2285,2024-04-04,1,1,0,1561,3.0,1
2286,2024-04-05,1,1,0,1561,3.0,1
2287,2024-04-06,1,1,0,1561,3.0,1


,t,a1,holiday,a2,a3,holidays,festival
2312,2024-05-01,1,1,0,1580,5.0,1
2313,2024-05-02,1,1,0,1580,5.0,1
2314,2024-05-03,1,1,0,1580,5.0,1
2315,2024-05-04,1,1,0,1580,5.0,1
2316,2024-05-05,1,1,0,1580,5.0,1


,t,a1,holiday,a2,a3,holidays,festival
2350,2024-06-08,1,1,0,1606,3.0,1
2351,2024-06-09,1,1,0,1606,3.0,1
2352,2024-06-10,1,1,0,1606,3.0,1


,t,a1,holiday,a2,a3,holidays,festival
2449,2024-09-15,1,1,0,1676,3.0,1
2450,2024-09-16,1,1,0,1676,3.0,1
2451,2024-09-17,1,1,0,1676,3.0,1


,t,a1,holiday,a2,a3,holidays,festival
2465,2024-10-01,1,1,0,1686,7.0,1
2466,2024-10-02,1,1,0,1686,7.0,1
2467,2024-10-03,1,1,0,1686,7.0,1
2468,2024-10-04,1,1,0,1686,7.0,1
2469,2024-10-05,1,1,0,1686,7.0,1
2470,2024-10-06,1,1,0,1686,7.0,1
2471,2024-10-07,1,1,0,1686,7.0,1


,t,a1,holiday,a2,a3,holidays_x,holidays_y
0,2018-01-01,1,1,0,0,1.0,1.0
1,2018-01-02,1,0,1,1,0.0,0.0
2,2018-01-03,1,0,1,2,0.0,0.0
3,2018-01-04,1,0,1,3,0.0,0.0
4,2018-01-05,1,0,1,4,2.0,2.0
...,...,...,...,...,...,...,...
2552,2024-12-27,1,0,1,1746,2.0,2.0
2553,2024-12-28,1,1,0,1746,2.0,2.0
2554,2024-12-29,1,1,0,1746,2.0,2.0
2555,2024-12-30,1,0,1,1747,0.0,0.0


In [147]:
df2['diff1'] = df2['t'].diff(1)
df2

,t,year,holiday,tx,workday,dayofweek,diff1
0,2018-01-01,2018,1,0,0,1,NaT
1,2018-01-02,2018,0,0,1,2,1 days
2,2018-01-03,2018,0,0,1,3,1 days
3,2018-01-04,2018,0,0,1,4,1 days
4,2018-01-05,2018,0,0,1,5,1 days
...,...,...,...,...,...,...,...
2552,2024-12-27,2024,0,0,1,5,1 days
2553,2024-12-28,2024,1,0,0,6,1 days
2554,2024-12-29,2024,1,0,0,7,1 days
2555,2024-12-30,2024,0,0,1,1,1 days


In [131]:
a1 = df[df['holiday'] ==1 ].groupby('a3')['t'].count().reset_index()
a1.t.value_counts()
df = df2[df2['holiday']==1].reset_index()

df['t1'] = df['t'].diff(1)
df['t2'] = df['t1'].dt.days
df['t3'] = df['t2'].diff().ne()
df

2     288
1      28
3      24
7      10
5       5
8       3
4       1
10      1
Name: t, dtype: int64

In [149]:
df2 = pd.read_csv('Data/df2.csv')
df2['t'] = pd.to_datetime(df2['t'])
df2['diff'] = df2['t'].diff(1)


In [154]:
len(pd.date_range(df2['t'].min(),df2['t'].max(),freq='1D'))

2557

((2557, 6), (2557, 7))

In [30]:
df = pd.DataFrame()
df['a'] = [1,2,3,4,5,6,6,6,]
df['b'] = df['a'].diff()
df['c'] = df['b'].ne(0)
df['d'] = df['c'].cumsum() 
df

,a,b,c,d
0,1,NaN,True,1
1,2,1.0,True,2
2,3,1.0,True,3
3,4,1.0,True,4
4,5,1.0,True,5
5,6,1.0,True,6
6,6,0.0,False,6
7,6,0.0,False,6


In [1]:
df

NameError: name 'df' is not defined